In [28]:
# %reset
from sympy import *
import numpy as np 
init_printing()

comp, out_of_plane = symbols( 'comp, OOP' )
iso = 1

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44 = symbols('D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44')
Exx, Ezz, Gxz, divV = symbols('Exx, Ezz, Gxz, divV')
stress_labels = ['Txx', 'Tzz', 'Txz', 'P']

if iso==1:
    # Isotropic
    Dv = Matrix( [ [D11, 0, 0, 0], [0, D22, 0, 0], [0, 0, D33, 0], [0, 0, 0, 0]  ] )
else:
    # Anisotropic
    Dv = Matrix( [ [D11, D12, D13, D14], [D21, D22, D23, D24], [D31, D32, D33, D34], [D41, D42, D43, D44]  ] )

# Check constitutive
Ed = Matrix( [ [Exx], [Ezz], [Gxz], [divV] ] )
T  = Dv*Ed
for i in range(4):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )

Txx = D11*Exx;
Tzz = D22*Ezz;
Txz = D33*Gxz;
P = 0;


In [29]:
# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')
dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])

# Stencil extension 1
uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE = symbols('uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE')
dofs_ext = Matrix([uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pW, pE, pSW, pSE, pNW, pNE = symbols('pW, pE, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pW, pE, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

# Flags
inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')
NeuN, NeuS, DirN, DirS, RegN, RegS  = symbols('NeuN, NeuS, DirN, DirS, RegN, RegS')
DirSW, DirSE, NeuSW, NeuSE = symbols('DirSW, DirSE, NeuSW, NeuSE')
DirNW, DirNE, NeuNW, NeuNE = symbols('DirNW, DirNE, NeuNW, NeuNE')

# North/South dVxdz
dVxdzN_Reg = (uN - uC )/dz
dVxdzS_Reg = (uC - uS )/dz
dVxdzN_Dir = ((-uC) - uC )/dz
dVxdzS_Dir = (uC - (-uC) )/dz
dVxdzN_Neu = ((uC) - uC )/dz
dVxdzS_Neu = (uC - (uC) )/dz

dVzdxN_Reg  = (vNE -vNW )/dx
dVzdxS_Reg  = (vSE -vSW )/dx

# Divergences
divW      = inW  *out_of_plane*( (uC  - uW )/dx + (vNW - vSW )/dz ) 
divE      = inE  *out_of_plane*( (uE  - uC )/dx + (vNE - vSE )/dz )
divSW     = inSWc*out_of_plane*( (uS  - uSW)/dx + (vSW - vSSW)/dz )
divSE     = inSEc*out_of_plane*( (uSE - uS )/dx + (vSE - vSSE)/dz )
divNW     = inNWc*out_of_plane*( (uN  - uNW)/dx + (vNNW- vNW )/dz )
divNE     = inNEc*out_of_plane*( (uNE - uN )/dx + (vNNE- vNE )/dz )

# Deviatoric normal strain rate
ExxW      = inW  *((uC -uW )/dx - comp*Rational(1,3)*divW) 
EzzW      = inW  *((vNW-vSW)/dz - comp*Rational(1,3)*divW)
ExxE      = inE  *((uE -uC )/dx - comp*Rational(1,3)*divE)
EzzE      = inE  *((vNE-vSE)/dz - comp*Rational(1,3)*divE)

# Additional missing stress tensor components needed for interpolation
GxzNE     = inNEv*((uNE - uE )/dz + ( vNEE - vNE  )/dx)
GxzSE     = inSEv*((uE  - uSE)/dz + ( vSEE - vSE  )/dx)
GxzNW     = inNWv*((uNW - uW )/dz + ( vNW  - vNWW )/dx)
GxzSW     = inSWv*((uW  - uSW)/dz + ( vSW  - vSWW )/dx)


# Shear strain rate
dVxdzN    = RegN*dVxdzN_Reg + DirS*dVxdzN_Dir + NeuS*dVxdzN_Neu
dVxdzS    = RegS*dVxdzS_Reg + DirN*dVxdzS_Dir + NeuN*dVxdzS_Neu
dVzdxN    = dVzdxN_Reg
dVzdxS    = dVzdxS_Reg

GxzN      = inN*(dVxdzN + dVzdxN)        
GxzS      = inS*(dVxdzS + dVzdxS) 
ExxNE     = inNEc*((uNE -uN )/dx - comp*Rational(1,3)*divNE)
ExxNW     = inNWc*((uN  -uNW)/dx - comp*Rational(1,3)*divNW)
ExxSE     = inSEc*((uSE -uS )/dx - comp*Rational(1,3)*divSE)
ExxSW     = inSWc*((uS  -uSW)/dx - comp*Rational(1,3)*divSW)
EzzNE     = inNEc*((vNNE-vNE)/dz - comp*Rational(1,3)*divNE)
EzzNW     = inNWc*((vNNW-vNW)/dz - comp*Rational(1,3)*divNW)
EzzSE     = inSEc*((vSE-vSSE)/dz - comp*Rational(1,3)*divSE)
EzzSW     = inSWc*((vSW-vSSW)/dz - comp*Rational(1,3)*divSW)
# INV 0
GxzE      = 0.25*( inN*GxzN + inS*GxzS + inNEv*GxzNE + inSEv*GxzSE)      # clear
GxzW      = 0.25*( inN*GxzN + inS*GxzS + inNWv*GxzNW + inSWv*GxzSW)
ExxN      = 0.25*( inE*ExxE + inW*ExxW + inNWc*ExxNW + inNEc*ExxNE)
ExxS      = 0.25*( inE*ExxE + inW*ExxW + inSWc*ExxSW + inSEc*ExxSE)
EzzN      = 0.25*( inE*EzzE + inW*EzzW + inNWc*EzzNW + inNEc*EzzNE) 
EzzS      = 0.25*( inE*EzzE + inW*EzzW + inSWc*EzzSW + inSEc*EzzSE)
pS        = 0.25*( inE*pE   + inW*pW   + inSWc*pSW   + inSEc*pSE)
pN        = 0.25*( inE*pE   + inW*pW   + inSWc*pNW   + inSEc*pNE)  

#----------------------------------------#
EdW  = Matrix([[ExxW], [EzzW], [GxzW], [pW]])
TW   = Dv*EdW
TxxW = TW[0].subs({D11:'D11W', D12:'D12W', D13:'D13W', D14:'D14W'})
#----------------------------------------#
EdE  = Matrix([[ExxE], [EzzE], [GxzE], [pE]])
TE   = Dv*EdE
TxxE = TE[0].subs({D11:'D11E', D12:'D12E', D13:'D13E', D14:'D14E'})
#----------------------------------------#
EdS  = Matrix([[ExxS], [EzzS], [GxzS], [pS]])
TS   = Dv*EdS
TxzS = TS[2].subs({D31:'D31S', D32:'D32S', D33:'D33S', D34:'D34S'})
#----------------------------------------#
EdN  = Matrix([[ExxN], [EzzN], [GxzN], [pN]])
TN   = Dv*EdN
TxzN = TN[2].subs({D31:'D31N', D32:'D32N', D33:'D33N', D34:'D34N'})
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS) - 1/dx*(pE - pW)
Fx  = -Fx

for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cUc)) + ';')

uW = (1.0/3.0)*D11W*inW*(OOP*comp*inW - 3)/pow(dx, 2);
uC = (1.0/3.0)*(3*pow(dx, 2)*(D33N*inN*(2*DirS + RegN) + D33S*inS*(2*DirN + RegS)) - pow(dz, 2)*(D11E*inE*(OOP*comp*inE - 3) + D11W*inW*(OOP*comp*inW - 3)))/(pow(dx, 2)*pow(dz, 2));
uE = (1.0/3.0)*D11E*inE*(OOP*comp*inE - 3)/pow(dx, 2);
uS = -D33S*RegS*inS/pow(dz, 2);
uN = -D33N*RegN*inN/pow(dz, 2);
vSW = ((1.0/3.0)*D11W*OOP*comp*pow(inW, 2) - D33S*inS)/(dx*dz);
vSE = (-1.0/3.0*D11E*OOP*comp*pow(inE, 2) + D33S*inS)/(dx*dz);
vNW = (-1.0/3.0*D11W*OOP*comp*pow(inW, 2) + D33N*inN)/(dx*dz);
vNE = ((1.0/3.0)*D11E*OOP*comp*pow(inE, 2) - D33N*inN)/(dx*dz);
uSW = 0;
uSE = 0;
uNW = 0;
uNE = 0;
vSWW = 0;
vSEE = 0;
vNWW = 0;
vNEE = 0;
vSSW = 0;
vSSE = 0;
vNNW = 0;
vNNE = 0;
pW = -1/dx;
pE = 1.0/dx;
pSW = 0;
pSE = 0;
pNW = 0;
pNE = 0;


In [30]:
# Compressibility switch
SxxBC = symbols('SxxBC') # boundary stress

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')
dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])

# Stencil extension 1
uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE = symbols('uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE')
dofs_ext = Matrix([uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pW, pE, pSW, pSE, pNW, pNE = symbols('pW, pE, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pW, pE, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

# Flags
inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')
NeuN, NeuS, DirN, DirS, RegN, RegS  = symbols('NeuN, NeuS, DirN, DirS, RegN, RegS')
DirSW, DirSE, NeuSW, NeuSE = symbols('DirSW, DirSE, NeuSW, NeuSE')
DirNW, DirNE, NeuNW, NeuNE = symbols('DirNW, DirNE, NeuNW, NeuNE')

# North/South dVxdz
dVxdzN_Reg = (uN - uC )/dz
dVxdzS_Reg = (uC - uS )/dz
dVxdzN_Dir = ((-uC) - uC )/dz
dVxdzS_Dir = (uC - (-uC) )/dz
dVxdzN_Neu = ((uC) - uC )/dz
dVxdzS_Neu = (uC - (uC) )/dz

dVzdxN_Reg  = (vNE -vNW )/dx
dVzdxN_DirW = (vNE -(-vNE) )/dx
dVzdxN_NeuW = (vNE -vNE )/dx
dVzdxN_DirE = (-vNW -vNW )/dx
dVzdxN_NeuE = (vNW -vNW )/dx
dVzdxS_Reg  = (vSE -vSW )/dx
dVzdxS_DirW = (vSE -(-vSE) )/dx
dVzdxS_NeuW = (vSE -vSE )/dx
dVzdxS_DirE = ((-vSW) -vSW )/dx
dVzdxS_NeuE = (vSW -vSW )/dx

# Divergences
divW      = inW  *out_of_plane*( (uC  - uW )/dx + (vNW - vSW )/dz ) 
divE      = inE  *out_of_plane*( (uE  - uC )/dx + (vNE - vSE )/dz )
divSW     = inSWc*out_of_plane*( (uS  - uSW)/dx + (vSW - vSSW)/dz )
divSE     = inSEc*out_of_plane*( (uSE - uS )/dx + (vSE - vSSE)/dz )
divNW     = inNWc*out_of_plane*( (uN  - uNW)/dx + (vNNW- vNW )/dz )
divNE     = inNEc*out_of_plane*( (uNE - uN )/dx + (vNNE- vNE )/dz )

# Deviatoric normal strain rate
ExxW      = inW  *((uC -uW )/dx - comp*Rational(1,3)*divW) 
EzzW      = inW  *((vNW-vSW)/dz - comp*Rational(1,3)*divW)
ExxE      = inE  *((uE -uC )/dx - comp*Rational(1,3)*divE)
EzzE      = inE  *((vNE-vSE)/dz - comp*Rational(1,3)*divE)

# Additional missing stress tensor components needed for interpolation
GxzNE     = inNEv*((uNE - uE )/dz + ( vNEE - vNE  )/dx)
GxzSE     = inSEv*((uE  - uSE)/dz + ( vSEE - vSE  )/dx)
GxzNW     = inNWv*((uNW - uW )/dz + ( vNW  - vNWW )/dx)
GxzSW     = inSWv*((uW  - uSW)/dz + ( vSW  - vSWW )/dx)


# Shear strain rate
dVxdzN    = RegN*dVxdzN_Reg + DirS*dVxdzN_Dir + NeuS*dVxdzN_Neu
dVxdzS    = RegS*dVxdzS_Reg + DirN*dVxdzS_Dir + NeuN*dVxdzS_Neu

dVzdxN    = DirNW*dVzdxN_DirW + NeuNW*dVzdxN_NeuW
dVzdxS    = DirSW*dVzdxS_DirW + NeuSW*dVzdxS_NeuW
# dVzdxN    = dVzdxN_Reg
# dVzdxS    = dVzdxS_Reg

GxzN      = inN*(dVxdzN + dVzdxN)        
GxzS      = inS*(dVxdzS + dVzdxS) 
ExxNE     = inNEc*((uNE -uN )/dx - comp*Rational(1,3)*divNE)
ExxNW     = inNWc*((uN  -uNW)/dx - comp*Rational(1,3)*divNW)
ExxSE     = inSEc*((uSE -uS )/dx - comp*Rational(1,3)*divSE)
ExxSW     = inSWc*((uS  -uSW)/dx - comp*Rational(1,3)*divSW)
EzzNE     = inNEc*((vNNE-vNE)/dz - comp*Rational(1,3)*divNE)
EzzNW     = inNWc*((vNNW-vNW)/dz - comp*Rational(1,3)*divNW)
EzzSE     = inSEc*((vSE-vSSE)/dz - comp*Rational(1,3)*divSE)
EzzSW     = inSWc*((vSW-vSSW)/dz - comp*Rational(1,3)*divSW)
# INV 0
GxzE      = 0.25*( inN*GxzN + inS*GxzS + inNEv*GxzNE + inSEv*GxzSE)      # clear
GxzW      = 0.25*( inN*GxzN + inS*GxzS + inNWv*GxzNW + inSWv*GxzSW)
ExxN      = 0.25*( inE*ExxE + inW*ExxW + inNWc*ExxNW + inNEc*ExxNE)
ExxS      = 0.25*( inE*ExxE + inW*ExxW + inSWc*ExxSW + inSEc*ExxSE)
EzzN      = 0.25*( inE*EzzE + inW*EzzW + inNWc*EzzNW + inNEc*EzzNE) 
EzzS      = 0.25*( inE*EzzE + inW*EzzW + inSWc*EzzSW + inSEc*EzzSE)
pS        = 0.25*( inE*pE   + inW*pW   + inSWc*pSW   + inSEc*pSE)
pN        = 0.25*( inE*pE   + inW*pW   + inSWc*pNW   + inSEc*pNE)  

#----------------------------------------#
EdW  = Matrix([[ExxW], [EzzW], [GxzW], [pW]])
TW   = Dv*EdW
TxxW = TW[0].subs({D11:'D11W', D12:'D12W', D13:'D13W', D14:'D14W'})
#----------------------------------------#
EdE  = Matrix([[ExxE], [EzzE], [GxzE], [pE]])
TE   = Dv*EdE
TxxE = TE[0].subs({D11:'D11E', D12:'D12E', D13:'D13E', D14:'D14E'})
#----------------------------------------#
EdS  = Matrix([[ExxS], [EzzS], [GxzS], [pS]])
TS   = Dv*EdS
TxzS = TS[2].subs({D31:'D31S', D32:'D32S', D33:'D33S', D34:'D34S'})
#----------------------------------------#
EdN  = Matrix([[ExxN], [EzzN], [GxzN], [pN]])
TN   = Dv*EdN
TxzN = TN[2].subs({D31:'D31N', D32:'D32N', D33:'D33N', D34:'D34N'})
#----------------------------------------#
Fx = 1/dx*(TxxE - TxxW) + 1/dz*(TxzN - TxzS) - 1/dx*(pE - pW)
Fx  = -Fx

# Stress boundary
SxxE = TxxE - pE
SxxW = 2*SxxBC - SxxE
Fx   = 1/dx*(SxxE - SxxW) + 1/dz*(TxzN - TxzS)
Fx   = -Fx/2

for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cUc)) + ';')



uW = 0;
uC = (1.0/6.0)*(-2*D11E*pow(dz, 2)*inE*(OOP*comp*inE - 3) + 3*pow(dx, 2)*(D33N*inN*(2*DirS + RegN) + D33S*inS*(2*DirN + RegS)))/(pow(dx, 2)*pow(dz, 2));
uE = (1.0/3.0)*D11E*inE*(OOP*comp*inE - 3)/pow(dx, 2);
uS = -1.0/2.0*D33S*RegS*inS/pow(dz, 2);
uN = -1.0/2.0*D33N*RegN*inN/pow(dz, 2);
vSW = 0;
vSE = (-1.0/3.0*D11E*OOP*comp*pow(inE, 2) + D33S*DirSW*inS)/(dx*dz);
vNW = 0;
vNE = ((1.0/3.0)*D11E*OOP*comp*pow(inE, 2) - D33N*DirNW*inN)/(dx*dz);
uSW = 0;
uSE = 0;
uNW = 0;
uNE = 0;
vSWW = 0;
vSEE = 0;
vNWW = 0;
vNEE = 0;
vSSW = 0;
vSSE = 0;
vNNW = 0;
vNNE = 0;
pW = 0;
pE = 1.0/dx;
pSW = 0;
pSE = 0;
pNW = 0;
pNE = 0;


In [31]:
# Compressibility switch
comp = symbols('comp')

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])

# Stencil extension 1
vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE = symbols('vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE')
dofs_ext = Matrix([vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pS, pN, pSW, pSE, pNW, pNE = symbols('pS, pN, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pS, pN, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

# Flags
inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')
DirE, DirW, NeuE, NeuW, RegE, RegW  = symbols('DirE, DirW, NeuE, NeuW, RegE, RegW')

# East/West velocity gradients
dVzdxE_Reg = (vE - vC )/dx
dVzdxW_Reg = (vC - vW )/dx
dVzdxE_Dir = (-vC - vC )/dx
dVzdxW_Dir = (vC - -vC )/dx
dVzdxE_Neu = (vC - vC )/dx
dVzdxW_Neu = (vC - vC )/dx

# Divergences
divS      = out_of_plane*( (uSE-uSW)/dx + (vC -vS )/dz )   
divN      = out_of_plane*( (uNE-uNW)/dx + (vN -vC )/dz )  
divSW     = out_of_plane*( (uSW-uSWW)/dx + (vW -vSW )/dz )
divSE     = out_of_plane*( (uSEE-uSE)/dx + (vE -vSE )/dz )
divNW     = out_of_plane*( (uNW-uNWW)/dx + (vNW -vW )/dz )
divNE     = out_of_plane*( (uNEE-uNE)/dx + (vNE -vE )/dz ) 

# Deviatoric normal strain
ExxS      = ((uSE-uSW)/dx - comp*Rational(1,3)*divS)
EzzS      = ((vC -vS )/dz - comp*Rational(1,3)*divS)
ExxN      = ((uNE-uNW)/dx - comp*Rational(1,3)*divN)
EzzN      = ((vN -vC )/dz - comp*Rational(1,3)*divN)

# Additional missing stress tensor components via interpolation
GxzSW     = ((uSW  - uSSW)/dz + (vS  - vSW)/dx)
GxzSE     = ((uSE  - uSSE)/dz + (vSE - vS )/dx)
GxzNW     = ((uNNW - uNW )/dz + (vN  - vNW)/dx)
GxzNE     = ((uNNE - uNE )/dz + (vNE - vN )/dx)

# Shear strain rate
GxzE       = inE*((uNE-uSE)/dz + RegE*dVzdxE_Reg + DirE*dVzdxE_Dir + NeuE*dVzdxE_Neu)
GxzW       = inW*((uNW-uSW)/dz + RegW*dVzdxW_Reg + DirW*dVzdxW_Dir + NeuW*dVzdxW_Neu)

# Additional missing stress tensor components for interpolation
ExxNE     = ((uNEE-uNE)/dx - comp*1/3*(divNE))
ExxNW     = ((uNW-uNWW)/dx - comp*1/3*(divNW))
ExxSE     = ((uSEE-uSE)/dx - comp*1/3*(divSE))
ExxSW     = ((uSW-uSWW)/dx - comp*1/3*(divSW))
EzzNE     = ((vNE -vE )/dz - comp*1/3*(divNE))
EzzNW     = ((vNW -vW )/dz - comp*1/3*(divNW))
EzzSE     = ((vE -vSE )/dz - comp*1/3*(divSE))
EzzSW     = ((vW -vSW )/dz - comp*1/3*(divSW))

# INV 0
GxzN      = Rational(1,4)*( inW*GxzW + inE*GxzE + inNWv*GxzNW + inNEv*GxzNE )
GxzS      = Rational(1,4)*( inW*GxzW + inE*GxzE + inSWv*GxzSW + inSEv*GxzSE ) 
ExxE      = Rational(1,4)*( inS*ExxS + inN*ExxN + inNEc*ExxNE + inSEc*ExxSE )
ExxW      = Rational(1,4)*( inS*ExxS + inN*ExxN + inNWc*ExxNW + inSWc*ExxSW )
EzzE      = Rational(1,4)*( inS*EzzS + inN*EzzN + inNEc*EzzNE + inSEc*EzzSE )
EzzW      = Rational(1,4)*( inS*EzzS + inN*EzzN + inNWc*EzzNW + inSWc*EzzSW )
pE        = Rational(1,4)*( inS*pS   + inN*pN   + inNEc*pNE   + inSEc*pSE )
pW        = Rational(1,4)*( inS*pS   + inN*pN   + inNWc*pNW   + inSWc*pSW )

#----------------------------------------#
EdS  = Matrix([[ExxS], [EzzS], [GxzS], [pS]])
TS   = Dv*EdS
TzzS = TS[1].subs({D21:'D21S', D22:'D22S', D23:'D23S', D24:'D24S'})
#----------------------------------------#
EdN  = Matrix([[ExxN], [EzzN], [GxzN], [pN]])
TN   = Dv*EdN
TzzN = TN[1].subs({D21:'D21N', D22:'D22N', D23:'D23N', D24:'D24N'})
#----------------------------------------#
EdW  = Matrix([[ExxW], [EzzW], [GxzW], [pW]])
TW   = Dv*EdW
TxzW = TW[2].subs({D31:'D31W', D32:'D32W', D33:'D33W', D34:'D34W'})
#----------------------------------------#
EdE  = Matrix([[ExxE], [EzzE], [GxzE], [pE]])
TE   = Dv*EdE
TxzE = TE[2].subs({D31:'D31E', D32:'D32E', D33:'D33E', D34:'D34E'})
#----------------------------------------#
Fz = 1/dz*(TzzN - TzzS) + 1/dx*(TxzE - TxzW) - (pN - pS)/dz 
Fz = -Fz
for i in range(len(dofs)):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cVc)) + ';') 

vW = -D33W*RegW*inW/pow(dx, 2);
vC = (1.0/3.0)*(-pow(dx, 2)*(D22N + D22S)*(OOP*comp - 3) + 3*pow(dz, 2)*(D33E*inE*(2*DirE + RegE) + D33W*inW*(2*DirW + RegW)))/(pow(dx, 2)*pow(dz, 2));
vE = -D33E*RegE*inE/pow(dx, 2);
vS = (1.0/3.0)*D22S*(OOP*comp - 3)/pow(dz, 2);
vN = (1.0/3.0)*D22N*(OOP*comp - 3)/pow(dz, 2);
uSW = ((1.0/3.0)*D22S*OOP*comp - D33W*inW)/(dx*dz);
uSE = (-1.0/3.0*D22S*OOP*comp + D33E*inE)/(dx*dz);
uNW = (-1.0/3.0*D22N*OOP*comp + D33W*inW)/(dx*dz);
uNE = ((1.0/3.0)*D22N*OOP*comp - D33E*inE)/(dx*dz);
vSW = 0;
vSE = 0;
vNW = 0;
vNE = 0;
uSWW = 0;
uSEE = 0;
uNWW = 0;
uNEE = 0;
uSSW = 0;
uSSE = 0;
uNNW = 0;
uNNE = 0;
pS = -1/dz;
pN = 1.0/dz;
pSW = 0;
pSE = 0;
pNW = 0;
pNE = 0;
